Importación de librerías

In [7]:
import pandas as pd
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

In [8]:
#data = pd.read_csv('lp5.csv', sep=',')
data = np.loadtxt('lp5.csv', delimiter=',',dtype=str)
data

array([['normal', '', '', '', '', ''],
       ['-2', '-1', '81', '0', '-5', '0'],
       ['-2', '-1', '79', '0', '-4', '0'],
       ...,
       ['-1', '1', '-3', '-3', '-2', '-3'],
       ['0', '-1', '-5', '-1', '1', '-3'],
       ['-1', '1', '4', '0', '-1', '-3']], dtype='<U18')

Separo las categorías y las matrices en listas por aparte

In [9]:
count = 0
matriz = []
label = []


for i in range(2624): 
    if (count==0):
        label.append((data[i]))
    if (count > 0):
        matriz.append((data[i]))
    count+=1 
    if (count == 16): 
      count = 0
print(label)
print(len(matriz))

[array(['normal', '', '', '', '', ''], dtype='<U18'), array(['normal', '', '', '', '', ''], dtype='<U18'), array(['normal', '', '', '', '', ''], dtype='<U18'), array(['normal', '', '', '', '', ''], dtype='<U18'), array(['normal', '', '', '', '', ''], dtype='<U18'), array(['normal', '', '', '', '', ''], dtype='<U18'), array(['normal', '', '', '', '', ''], dtype='<U18'), array(['normal', '', '', '', '', ''], dtype='<U18'), array(['normal', '', '', '', '', ''], dtype='<U18'), array(['normal', '', '', '', '', ''], dtype='<U18'), array(['normal', '', '', '', '', ''], dtype='<U18'), array(['normal', '', '', '', '', ''], dtype='<U18'), array(['normal', '', '', '', '', ''], dtype='<U18'), array(['normal', '', '', '', '', ''], dtype='<U18'), array(['normal', '', '', '', '', ''], dtype='<U18'), array(['normal', '', '', '', '', ''], dtype='<U18'), array(['normal', '', '', '', '', ''], dtype='<U18'), array(['normal', '', '', '', '', ''], dtype='<U18'), array(['normal', '', '', '', '', ''], dtype='

Añado los datos de la lista "matriz" a una matriz

In [10]:
matriz_data=np.zeros((len(matriz), 6))

for i in range(2460):
    for j in range(6):
        matriz_data[i][j] = float(matriz[i][j])

Uno la matriz y los labels en una sola matriz

In [11]:
count = 0
mat_final=np.zeros((164,15,6))
for i in range(164): 
    for j in range(15):
        for k in range(6):
            mat_final[i][j][k] = matriz[count][k] 
        count += 1 



Categorización de los labels para la salida del clasificador

In [1]:
cat = pd.DataFrame(label, columns=['category', '1', '2', '3', '4', '5'])
categorias = {
    'normal':[1, 0, 0, 0, 0],
    'collision_in_tool':[0, 1, 0, 0, 0],
    'collision_in_part':[0, 0, 1, 0, 0],
    'bottom_collision':[0, 0, 0, 1, 0],
    'bottom_obstruction':[0, 0, 0, 0, 1]}

cat = cat.replace({'label':categorias})

display(cat)
cat[['1', '2', '3', '4', '5']] = cat['label'].str.split(' ', 4, expand= True)
cat = cat[['1', '2', '3', '4', '5']].astype(int)


NameError: name 'pd' is not defined